# Mesonet Testing

This notebook derives ET values from the created data product and compares them with the existing Mesonet Stations and their PM-derived ET.

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [12]:
import os
import time
import random
import itertools
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import pickle
import joblib
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool

import phik
from phik.report import plot_correlation_matrix
from phik import report

from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from tqdm.notebook import tqdm

from src.PM_eq import penman_monteith
from src.process_fluxes import create_rolling_windows

In [22]:
mod = np.load('../data_v02/validation/MOD_features.npy', allow_pickle=True)
mod_idxs = mod[:,-1].astype(int)
mod = mod[:,:-1] # drop last (idx)
era5 = np.load('../data_v02/validation/ERA5_features.npy', allow_pickle=True)
era5_idxs = era5[:,0,-1].astype(int)
era5 = era5[:,:,:-1] # drop last (idx)

fluxes = pd.read_csv('../data_v02/validation/validation_dataset_IGBP.csv')
#fluxes = fluxes[fluxes['LE_F_MDS_QC']>=0.75]
keep_idxs = [idx for idx in fluxes.index.tolist() if (idx in era5_idxs) and (idx in mod_idxs)]
fluxes = fluxes.loc[keep_idxs,:]
fluxes['TIMESTAMP'] = pd.to_datetime(fluxes.TIMESTAMP, format='%Y-%m-%d')
#targets = fluxes.LE_F_MDS.values

fluxes

,TIMESTAMP,ETos_daily_in,ETrs_daily_in,site,lat,lon,IGBP
0,2019-01-01,5.04875,6.49125,Becker,45.34399,-93.85014,CRO
1,2019-01-02,9.37625,13.70375,Becker,45.34399,-93.85014,CRO
2,2019-01-03,16.58875,23.80125,Becker,45.34399,-93.85014,CRO
3,2019-01-04,17.31000,22.35875,Becker,45.34399,-93.85014,CRO
4,2019-01-05,18.03125,25.96500,Becker,45.34399,-93.85014,CRO
...,...,...,...,...,...,...,...
18278,2022-12-27,9.37625,12.98250,Westport,45.71509,-95.17179,CRO
18279,2022-12-28,6.49125,7.21250,Westport,45.71509,-95.17179,CRO
18280,2022-12-29,2.88500,2.88500,Westport,45.71509,-95.17179,CRO
18281,2022-12-30,7.93375,9.37625,Westport,45.71509,-95.17179,CRO


#TODO: compare with time series from the data product once it's available!